## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [1]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c

float64
7.89 ms ± 76.5 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 64.2 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.6 ms ± 25.6 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


a) Usando la librería Cupy

In [3]:
import numpy as np
import cupy as cp
from cupyx.profiler import benchmark

# Constantes
size = 5_000_000
a, b, c = 3.5, 2.8, 10.0

# Arrays en CPU
x_cpu = np.random.rand(size).astype(np.float64)
y_cpu = np.random.rand(size).astype(np.float64)
z_cpu = a*x_cpu**2 + b*y_cpu + c

# Caso 1: copiando desde CPU
def grade2_cupy_copy(x, y, a, b, c):
    x_gpu = cp.asarray(x)
    y_gpu = cp.asarray(y)
    return a*x_gpu**2 + b*y_gpu + c

execution_copy = benchmark(grade2_cupy_copy, (x_cpu, y_cpu, a, b, c), n_repeat=10, n_warmup=2)
gpu_avg_copy_ms = np.mean(execution_copy.gpu_times) * 1000 #milisegundos
print(f"Tiempo promedio GPU copiando desde CPU: {gpu_avg_copy_ms:.3f} ms")

# Caso 2: arrays ya en GPU
x_gpu = cp.random.rand(size, dtype=cp.float64)
y_gpu = cp.random.rand(size, dtype=cp.float64)

def grade2_cupy_no_copy(x_gpu, y_gpu, a, b, c):
    return a*x_gpu**2 + b*y_gpu + c

execution_no_copy = benchmark(grade2_cupy_no_copy, (x_gpu, y_gpu, a, b, c), n_repeat=10, n_warmup=2)
gpu_avg_no_copy_ms = np.mean(execution_no_copy.gpu_times) * 1000 #milisegundos
print(f"Tiempo promedio GPU arrays directos en GPU: {gpu_avg_no_copy_ms:.3f} ms")

Tiempo promedio GPU copiando desde CPU: 14.628 ms
Tiempo promedio GPU arrays directos en GPU: 4.524 ms


b) Usando la librería Numba

In [4]:
import numpy as np
import cupy as cp
from numba import vectorize, float64
from cupyx.profiler import benchmark

# Constantes
a = 3.5
b = 2.8
c = 10.0
size = 5_000_000

# Crear la ufunc de Numba para GPU
@vectorize([float64(float64, float64, float64, float64, float64)], target='cuda')
def grade2_numba(x, y, a, b, c):
    return a*x*x + b*y + c

# Copiando datos desde CPU
x_cpu = np.random.rand(size).astype(np.float64)
y_cpu = np.random.rand(size).astype(np.float64)

# Copiar manualmente a GPU para no contar el overhead en la medición
x_gpu = cp.asarray(x_cpu)
y_gpu = cp.asarray(y_cpu)
z_gpu = cp.zeros_like(x_gpu)

# Benchmark
def run_copy():
    z = grade2_numba(x_gpu, y_gpu, a, b, c)

result_copy = benchmark(run_copy, n_repeat=10, n_warmup=2)
gpu_avg_copy_ms = np.mean(result_copy.gpu_times) * 1000
print(f"Tiempo promedio GPU copiando desde CPU: {gpu_avg_copy_ms:.3f} ms")

# Arrays creados directamente en GPU
x_gpu_direct = cp.random.rand(size, dtype=cp.float64)
y_gpu_direct = cp.random.rand(size, dtype=cp.float64)
z_gpu_direct = cp.zeros_like(x_gpu_direct)

# Benchmark
def run_direct():
    z = grade2_numba(x_gpu_direct, y_gpu_direct, a, b, c)

result_direct = benchmark(run_direct, n_repeat=10, n_warmup=2)
gpu_avg_direct_ms = np.mean(result_direct.gpu_times) * 1000
print(f"Tiempo promedio GPU arrays directos en GPU: {gpu_avg_direct_ms:.3f} ms")

Tiempo promedio GPU copiando desde CPU: 2.450 ms
Tiempo promedio GPU arrays directos en GPU: 2.321 ms


c) Los resultados muestran que la operación vectorizada (z = a x^2 + b y + c) se ejecuta muy rápidamente en GPU. Usando CuPy, copiar los arrays desde CPU introduce un overhead significativo, aumentando el tiempo de 4.524 ms (solo kernel) a 14.628 ms, mientras que al usar arrays ya en GPU el tiempo se reduce notablemente, mostrando que la GPU procesa la operación de manera eficiente. Usando Numba, la diferencia entre copiar desde CPU y usar arrays en GPU es mínima (2.450 ms y 2.321 ms;se sigue obteniendo un mayor tiempo debido al copiado) ya que la librería maneja automáticamente la transferencia de datos de manera optimizada y compila la función a código de bajo nivel. En general, la principal limitación en los tiempos totales es la transferencia de memoria desde la CPU, mientras que la ejecución de la función en GPU es prácticamente instantánea.